## Full spectral fitting of a passively evolving galaxy;

This notebook will demosntrate how we get redshift of a passively evoluving (i.e. no emission lines) galaxy with NIRISS spectra+photometry and HST photometry.


In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import ascii,fits
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.table import QTable
import astropy.units as u

from astropy import __version__ as asver
asver

In [ ]:
# https://github.com/mtakahiro/gsf/tree/version1.4
import gsf
print(gsf.__version__)

from gsf.function import get_input
from gsf.gsf import run_gsf_template
from gsf.plot_sed_logA import plot_sed, plot_corner_physparam_frame, plot_corner_physparam_summary
from gsf.plot_sfh_logA import plot_sfh

### Setup gsf

In [ ]:
# Initial setup for gsf.

# Data directory;
DIR_DATA = './data_niriss/'


# Auto load input dictionary;
inputs = get_input()

# change Z;
# Flag;
fplt = 0
inputs['DIR_TEMP'] = './templates/nonebular_miles/'

# Output directory;
inputs['DIR_OUT'] = './output/'

# If templates exit already, then let's save time.
# (But if you changed metallicity range or age pixels, fplt needs to be 0.)
if os.path.exists('%s/spec_all.asdf'%inputs['DIR_TEMP']):
    fplt = 1

inputs['ID'] = '3'

# Initial guess of redshift, or the true value if known. 
# We will later do redshift fit later, though.
inputs['ZGAL'] = 2.0

# Redshift as a free parameter?
inputs['ZMC'] = 1


# Metallicity range, in logZsun;
inputs['ZMIN'] = -0.8
inputs['ZMAX'] = 0.624
inputs['DELZ'] = 0.2
# You can fix metallicity;
#inputs['ZFIX'] = 0.0


# Templates;
inputs['BPASS'] = 0
inputs['AGE'] = '0.01,0.03,0.1,0.3,0.5,0.7,1.0,1.5,2.0,3.0'
# You can fix age;
#inputs['AGEFIX'] = '0.3' # '0.1,0.3,0.5'


# Data;
DIR_EXTR = DIR_DATA
spec_file = 'l3_nis_G150C_s00003_1d_cont_fnu.txt'
inputs['DIR_EXTR'] = DIR_EXTR
inputs['SPEC_FILE'] =  spec_file
inputs['DIR_FILT'] = './filter/'
inputs['CAT_BB'] = '%s/l3_nis_flux.cat'%DIR_DATA

# Filters;
# Each number corresponds to EAZY's filter ids. See also filter/filt_Sep20.lst
# These numbers need to be found in inputs['CAT_BB'] file.
inputs['FILTER'] = '309,310,311,308,1,4,6,202,203,204,205'


# Morphology convolution; Necessary for NIRISS spectra;
filt = 'f200w'
inputs['MORP'] = 'moffat'
inputs['MORP_FILE'] = 'l3_nis_f200w_G150C_s00003_moffat.txt'


# MCMC part;
inputs['NCPU'] = 1 # For notebook, somehow multiprocessing causes error. So set to 1.
inputs['NMC'] = 1000 # NMC for the main SED fit
inputs['NMCZ'] = 30 # NMC for the redshift fit


# Visual inspection;
# Set to 0 (False), as Notebook cannot show actively iterating plot;
inputs['ZVIS'] = 0


# Emission line masking;
#LW = [3727, 4341, 4861, 4960, 5008, 6563, 6717, 6731] # in AA, rest.
#inputs['LINE'] = LW


# Initial fit:
inputs['FNELD'] = 1

In [ ]:
# Then, run template generate function;
mb = run_gsf_template(inputs, fplt=fplt)
fplt = 1


In [ ]:
# You can write down the input file in an ascii file.
from gsf.function import write_input
write_input(inputs, file_out='gsf.input')


In [ ]:
# Do a quick fit at z=z_guess;
mb.zprev = mb.zgal
#f_add = mb.add_param(mb.fit_params, sigz=mb.sigz, zmin=mb.zmin, zmax=mb.zmax)
out, chidef, Zbest, fm_tmp, xm_tmp = mb.quick_fit(mb.zgal, mb.Cz0, mb.Cz1, f_get_templates=True)


In [ ]:
flag_z = mb.fit_redshift(xm_tmp, fm_tmp)


### Now, let's improve the fit by finding the true redshift;

In [ ]:
# Preparing Fitting Spectral Template from the library generated above;
# Here, we use 5 templates for find redshift;

dict = mb.read_data(mb.Cz0, mb.Cz1, mb.zgal)
ages = [0.01,0.03,0.1,0.3,1.0]
ntmp = len(ages)

for nn in range(ntmp):
    # For simplicity, no dust attenuation (Av=0), Z fixed to solar (Z=0).
    flux_all, wave_all = mb.fnc.get_template(mb.lib_all, Amp=1.0, T=ages[nn], Av=0.0, Z=0.0, zgal=mb.zgal)
    
    con_tmp = (1000 < wave_all / (1.+mb.zgal)) & (wave_all / (1.+mb.zgal) < 60000)

    # Don't forget to blueshift the template.
    xm_tmp = wave_all[con_tmp] / (1.+mb.zgal)
    fm_tmp = flux_all[con_tmp]

    if nn == 0:
        fm_tmps = np.zeros((ntmp,len(xm_tmp)),'float')

    fm_tmps[nn,:] = fm_tmp[:]


In [ ]:
# Then, run redshift fitting; 
# dict : dictionary that includes a lot of things, including data.
# zliml, zlimu : Redshift search range, lower and upper limits.

# This should not be too small, if z-distribution is used as prior.
delzz = 0.1
zspace, chi2s = mb.search_redshift(dict, xm_tmp, fm_tmps, zliml=1., zlimu=4., delzz=delzz)


In [ ]:
# Plot;
plt.plot(zspace,chi2s[:,1])
plt.ylabel('$\chi^2$',fontsize=18)
plt.xlabel('$z$',fontsize=18)
plt.title('Redshift Fitting Result')


In [ ]:
# Get z at the chi2 minimum.
izfit = np.argmin(chi2s[:,1])
zfit = zspace[izfit]
print('zfit is %.2f'%(zfit))


In [ ]:
# Use chi2 as a prior
# User can provide phot-z prob by EAZY too.
prior = {}
prior['z'] = zspace
prior['chi2'] = chi2s[:,1]


In [ ]:
# Or arbitrary prior;
if False:
    # Or define a new prior:
    zspace_tmp = np.arange(0,13,0.01)
    chi2s_tmp = zspace_tmp * 0 + 99
    con_tmp = (zspace_tmp>1.8) & (zspace_tmp<2.1)
    chi2s_tmp[con_tmp] = 1.0

    prior = {}
    prior['z'] = zspace_tmp
    prior['chi2'] = chi2s_tmp
    

In [ ]:
# Repeat the quick fit at the proposed redshift;
inputs['ZGAL'] = zfit
inputs['NMCZ'] = 30

# Update with a new z input
mb = run_gsf_template(inputs, fplt=fplt)

mb.zprev = mb.zgal
out, fm_tmp, xm_tmp = mb.quick_fit(mb.zgal, mb.Cz0, mb.Cz1)


In [ ]:
out

### Now the result looks good

In [ ]:
plt.close()

# Plot the result;
flux_all, wave_all = mb.fnc.tmp04(out, f_val=True, lib_all=True)

# Template
plt.errorbar(wave_all, flux_all, ls='-', color='b', zorder=0, label='Fit')

# plot;
plt.scatter(dict['xbb'], dict['fybb'], marker='o', c='orange', edgecolor='k', s=150, zorder=2, alpha=1, label='Broadband')

if True: # Spec data;
    plt.errorbar(dict['x'], dict['fy'], yerr=dict['ey'], ls='', color='gray', zorder=1, alpha=0.3)
    plt.scatter(dict['x'], dict['fy'], marker='o', color='r',edgecolor='r', s=10, zorder=1, alpha=1, label='Spectrum')

plt.xlim(3000,30000)
#plt.ylim(0,25)
plt.xscale('log')

plt.legend(loc=2)
plt.xlabel('Wavelength (AA)')
plt.ylabel('Flux (MJy/sr)')


### Now fit redshift in more details;

In [ ]:
dict = mb.read_data(mb.Cz0, mb.Cz1, mb.zgal)

# By usinng the bbest fit template above;
con_tmp = ()
xm_tmp = wave_all[con_tmp]
fm_tmp = flux_all[con_tmp]

# Update inputs; 
inputs['NMCZ'] = 300
inputs['NWALKZ'] = 30
mb.update_input(inputs)

# Redshift fit when BB photometry only?
f_bb_zfit = True

# This works only when spectrum is provided.
mb.fit_redshift(xm_tmp, fm_tmp, delzz=0.01, zliml=2., zlimu=2.55, ezmin=0.01, snlim=0, \
                f_bb_zfit=f_bb_zfit, priors=prior)


In [ ]:
# This is normalization;
# Should be ~1, as we have already normalized the spectra to BB fluxes.
print('Redshift 16/50/84th percentile range :',mb.z_cz)
print(mb.Czrec0)
print(mb.Czrec1)

### Now, run the whole SED fitting;

In [ ]:
# No interactive fit;
inputs['ZMC'] = 1
inputs['ZVIS'] = 0
inputs['NMC'] = 1000
inputs['ZGAL'] = mb.z_cz[1]

# Update inputs; 
mb.update_input(inputs)

# Since already z-fit done, we can skip z-fit;
skip_fitz = True

# Main;
flag_suc = mb.main(cornerplot=True, specplot=1, sigz=1.0, ezmin=0.01, ferr=0, f_move=False, skip_fitz=skip_fitz, out=out)


In [ ]:
# Plot SFH;

# Plot Starforming Main Sequence from Speagle+14?
f_SFMS = True
f_symbol = True
skip_zhist = True
tau_lim = 0.01
tset_SFR_SED = 10
mmax = 300

plot_sfh(mb, f_comp=mb.ftaucomp, fil_path=mb.DIR_FILT, mmax=mmax,
inputs=mb.inputs, dust_model=mb.dust_model, DIR_TMP=mb.DIR_TMP, f_silence=True, 
f_SFMS=f_SFMS, f_symbol=f_symbol, skip_zhist=skip_zhist, tau_lim=tau_lim, tset_SFR_SED=tset_SFR_SED)


In [ ]:
# Plot SED;
plot_sed(mb, fil_path=mb.DIR_FILT,
figpdf=False, save_sed=True, inputs=mb.inputs, mmax=300,
f_fill=True, dust_model=mb.dust_model, DIR_TMP=mb.DIR_TMP, f_label=True)


In [ ]:
# Physical parameters;
plot_corner_physparam_summary(mb)